<a href="https://colab.research.google.com/github/diegompin/mtsa/blob/feature%2Fv0.0.8/examples/MTSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MTSA - **M**ultiple **T**ime **S**eries **A**nalysis


### Installing MTSA module:

In [ ]:
#!pip install mtsa

### Cloning MTSA repository:

*Obs: This step is necessary to access the example data.*

In [ ]:
#!git clone https://github.com/diegompin/mtsa.git

### Setting data directory:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
path_input_1 = os.path.join(os.getcwd(),  "sample_data", "machine_type_1", "id_00")
path_input_2 = os.path.join(os.getcwd(),  "mtsa", "examples", "sample_data", "machine_type_1", "id_00")

In [3]:
import torch
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
assert len(gpus) > 0, "Not enough GPU hardware devices available"
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

2024-04-02 21:51:16.558818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 21:51:17.176662: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-02 21:51:17.176705: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-04-02 21:51:17.176711: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [4]:
import sys
sys.path.append("..")

### Reading Data Files:

In [5]:
from mtsa import calculate_aucroc

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [6]:

from mtsa import files_train_test_split
#path_input_1 = '/data/MIMII/fan/id_00/'
X_train, X_test, y_train, y_test = files_train_test_split(path_input_1)
if(len(y_train) == 0): 
    X_train, X_test, y_train, y_test = files_train_test_split(path_input_2)
y_train

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [7]:
from mtsa.models.ganf import GANF

model_GANF = GANF(sampling_rate=10)

In [9]:
model_GANF.fit(X_train, y_train)
model_GANF.model

KeyboardInterrupt: 

In [ ]:
auc = calculate_aucroc(model_GANF, X_test, y_test)
auc

0.36

### MFFCMix Model:

In [23]:
from mtsa import MFCCMix
model_mfccmix = MFCCMix()
model_mfccmix.fit(X_train, y_train)
model_mfccmix.model

Pipeline(steps=[('wav2array', Wav2Array()),
                ('array2mfcc', Array2Mfcc(sampling_rate=None)),
                ('features',
                 FeatureUnion(transformer_list=[('M',
                                                 MagnitudeMeanFeatureMfcc()),
                                                ('S',
                                                 MagnitudeStdFeatureMfcc()),
                                                ('C',
                                                 CorrelationFeatureMfcc())])),
                ('final_model', GaussianMixture())])

In [24]:
auc = calculate_aucroc(model_mfccmix, X_test, y_test)
auc

0.92

### Hitachi Model:

In [28]:
from mtsa import Hitachi
model_hitachi = Hitachi()
model_hitachi.fit(X_train, y_train)
model_hitachi.model

2024-04-02 00:35:12.036458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-02 00:35:12.036717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-02 00:35:12.036925: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Pipeline(steps=[('wav2array', Wav2Array(mono=False)),
                ('demux2array', Demux2Array()),
                ('array2melspec',
                 Array2MelSpec(frames=5, hop_length=512, n_fft=1024, n_mels=64,
                               power=2.0, sampling_rate=None)),
                ('final_model',
                 <mtsa.models.hitachi.AutoEncoderMixin object at 0x7fa527ed1700>)])

In [29]:
auc = calculate_aucroc(model_hitachi, X_test, y_test)
auc

14/14 [==============================] - 0s 925us/step


0.27999999999999997